In [ ]:
# Import all requried libraries
# Define the Class for the laser
# Define sound function
# Need to set up GPIO
# Establish pigpio port

# Start up the Hand ( play music and startup sounds )
# Wait for fire button to be 